# Setup

In [1]:
!python -m pip -q install --upgrade pip
!pip install -e .

Obtaining file:///home/pprados/workspace.bda/langchain-references
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for langchain_references (pyproject.toml) ... done
  Created wheel for langchain_references: filename=langchain_references-0.0.0-0.editable-py3-none-any.whl size=7019 sha256=736a538956f99d092206fdf1a5888fcaed6c25e3a6f0c7a1e980b4d603b95949
  Stored in directory: /tmp/pip-ephem-wheel-cache-k98e9tq2/wheels/a4/32/87/e1597a08c9777b23e5130be598110a6cbb924fc8978b83f36d
Successfully built langchain_references
  Attempting uninstall: langchain_references
    Found existing installation: langchain_references 0.0.0
    Not uninstalling langchain-references at /home/pprados/workspace.bda/langchain-references, outside environment /home/pprados/workspace.bda/langchain-rag/.venv
    Can't uninstall 'langchain_re

In [2]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain-references
%pip install -qU langchain-community
%pip install -qU langchain-text-splitters

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import langchain_references

langchain_references.__version__

'0.0.0'

# Document loading, retrieval methods and text splitting
Load documents from the web and split them into smaller chunks for processing.

In [4]:
import os
os.environ["USER_AGENT"] = "langhchain-references"

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Choose a vectorstore to use for similarity search

In [5]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

Choose a model to use for question answering

In [6]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3",
)

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
)

Combine the documents into a single string, but with a uniq small numeric id.

In [7]:
def format_docs(docs):
    # return "\n\n".join(doc.page_content for doc in docs)
    return "\n".join(
        # Add a document id so that LLM can reference it 
        [f"<document id={i + 1}>\n{doc.page_content}\n</document>\n" for i, doc in
         enumerate(docs)]
    )


# Manage references with langchain-reference

Create a prompt with {format_references} and {context} placeholders.

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved documents to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

{format_references}
  
<documents>
{documents}
</documents>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

Create a context with documents and format_references.

In [9]:
from langchain_references import *

context = RunnablePassthrough.assign(
    documents=lambda input: format_docs(input["documents"]),
    format_references=lambda _: FORMAT_REFERENCES,
)

Create a chain with the context, rag_prompt and model, and encapulate it with `manage_references()`.

In [10]:
from langchain_core.output_parsers import StrOutputParser

chain = manage_references(
    context
    | rag_prompt
    | model,
) | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

Invoke the chain with the documents and question.

In [11]:
print(chain.invoke({"documents": docs, "question": question}))

According to documents ^[1](https://lilianweng.github.io/posts/2023-06-23-agent/)^ and ^[1](https://lilianweng.github.io/posts/2023-06-23-agent/)^ there are three approaches to task decomposition: using simple prompting like "Steps for XYZ.\n1.", using task-specific instructions, and with human inputs. These methods help break down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.

- **1** [LLM Powered Autonomous Agents | Lil'Log](https://lilianweng.github.io/posts/2023-06-23-agent/)

